In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [31]:
with open('reduce_mem_train_transaction.pkl', 'rb') as f:
    train_data = pickle.load(f)
    for value,typeof in zip(train_data.columns.values,train_data.ftypes.values):
        print((value,typeof))
    print (len(train_data.columns))
with open('reduce_mem_test_transaction.pkl', 'rb') as f:
    test_data = pickle.load(f)

('TransactionID', 'int32:dense')
('isFraud', 'int8:dense')
('TransactionDT', 'int32:dense')
('TransactionAmt', 'float16:dense')
('ProductCD', 'object:dense')
('card1', 'int16:dense')
('card2', 'float16:dense')
('card3', 'float16:dense')
('card4', 'object:dense')
('card5', 'float16:dense')
('card6', 'object:dense')
('addr1', 'float16:dense')
('addr2', 'float16:dense')
('dist1', 'float16:dense')
('dist2', 'float16:dense')
('P_emaildomain', 'object:dense')
('R_emaildomain', 'object:dense')
('C1', 'float16:dense')
('C2', 'float16:dense')
('C3', 'float16:dense')
('C4', 'float16:dense')
('C5', 'float16:dense')
('C6', 'float16:dense')
('C7', 'float16:dense')
('C8', 'float16:dense')
('C9', 'float16:dense')
('C10', 'float16:dense')
('C11', 'float16:dense')
('C12', 'float16:dense')
('C13', 'float16:dense')
('C14', 'float16:dense')
('D1', 'float16:dense')
('D2', 'float16:dense')
('D3', 'float16:dense')
('D4', 'float16:dense')
('D5', 'float16:dense')
('D6', 'float16:dense')
('D7', 'float16:dense')

In [3]:
def make_hour_feature(df, tname='TransactionDT'):
    hours = df[tname] / (3600)        
    encoded_hours = np.floor(hours) % 24
    return encoded_hours  

In [4]:
# 将时间信息组织为特征值   类似的  邮箱
'''
由图可知  高低类别间划分区分度不明显 如果将时间信息划分为几类  信息损失较大
'''
train_data['hours'] = make_hour_feature(train_data)
hoursX = train_data.groupby('hours').mean()['isFraud']
hoursC = train_data.groupby('hours')['isFraud'].count()
# print(train_data['hours'].astype("int").values)
hoursX = hoursX.values
hoursCs= (hoursC[np.array(train_data['hours'].astype("int").values)]-hoursC.min())/(hoursC.max()-hoursC.min()) 
print(np.unique(np.array(hoursCs)))
train_data['hours'] = (hoursX[np.array(train_data['hours'].astype("int").values)]-hoursX.min())/(hoursX.max()-hoursX.min()) 
print(np.unique(np.array(train_data['hours'])))

[0.         0.00282571 0.02896357 0.03090625 0.08900999 0.10969825
 0.18220809 0.25158946 0.31183772 0.44999495 0.46228176 0.61189323
 0.65215965 0.76491069 0.79170451 0.89100817 0.91379049 0.94535271
 0.96488041 0.97537592 0.98799072 0.98804117 0.99159855 1.        ]
[0.         0.0159452  0.03016343 0.07956877 0.09073006 0.10123957
 0.10203239 0.1038384  0.11782609 0.13357994 0.13680285 0.142392
 0.14831875 0.16953898 0.17537942 0.18535943 0.19140231 0.34851693
 0.36440084 0.56977994 0.65919734 0.80596692 0.84272427 1.        ]


In [5]:
train_data['hours']=np.sqrt(np.square(train_data['hours'].values+np.square(hoursCs.values)))
print(np.unique(np.array(train_data['hours'])))

[0.15402732 0.20249546 0.20343602 0.36523973 0.39906386 0.44125742
 0.4457597  0.54979274 0.60297973 0.65695946 0.66712012 0.68632793
 0.80596692 0.84273225 0.89592796 0.91458184 1.0009552  1.03483261
 1.06323072 1.06918427 1.1098053  1.12007053 1.12444441 1.142392  ]


In [6]:
train_data['hours'] = (train_data['hours']-train_data['hours'].min())/(train_data['hours'].max()-train_data['hours'].min())
print(np.unique(np.array(train_data['hours'])))

[0.         0.04903872 0.04999036 0.21369886 0.24792119 0.29061145
 0.29516674 0.40042449 0.45423761 0.50885281 0.51913308 0.53856701
 0.65961443 0.69681257 0.75063451 0.76950799 0.85689816 0.89117439
 0.91990681 0.92593045 0.96702968 0.97741575 0.98184112 1.        ]


In [7]:
# 统计card的类别
cardTypesObj = {}
isFirstCir = True
for i in range(1,7):
    datas=train_data['card'+str(i)]
    if datas.dtypes=='object':
        cardTypesObj['card'+str(i)]=datas.unique()
    else:
        cardTypesObj['card'+str(i)]=['1',None]

In [8]:
print(cardTypesObj)
# def getEveryCardTypeName(i,obj,retObj,endNum=7):
#     if i<endNum:
#         for item in obj:
#             retObj['card'+str(i)]=item
#             getEveryCardTypeName(i+1,obj,retObj)
#     else:
        
#         return 

{'card1': ['1', None], 'card2': ['1', None], 'card3': ['1', None], 'card4': array(['discover', 'mastercard', 'visa', 'american express', nan],
      dtype=object), 'card5': ['1', None], 'card6': array(['credit', 'debit', nan, 'debit or credit', 'charge card'],
      dtype=object)}


In [9]:
for i in range(1,7):
    if train_data['card'+str(i)].dtypes!='object':
        train_data['card'+str(i)]=train_data['card'+str(i)].where(train_data['card'+str(i)]>0,0)
        train_data['card'+str(i)]=train_data['card'+str(i)].mask(train_data['card'+str(i)]>0,1)
        print(train_data['card'+str(i)].unique())
    else:
        print(train_data['card'+str(i)].unique())


[1]
[0. 1.]
[1. 0.]
['discover' 'mastercard' 'visa' 'american express' nan]
[1. 0.]
['credit' 'debit' nan 'debit or credit' 'charge card']


In [10]:
datas=train_data[['TransactionID','card1','card2','card3','card4','card5','card6']]
print(datas.groupby(['card1','card2','card3','card4','card5','card6']).count())

                                                          TransactionID
card1 card2 card3 card4            card5 card6                         
1     0.0   1.0   american express 0.0   credit                      56
                  discover         0.0   credit                      10
                                   1.0   credit                    1238
                                         debit                       69
                  mastercard       0.0   credit                     114
                                         debit                        8
                                   1.0   credit                      19
                                         debit                       49
                  visa             0.0   credit                     346
                                         debit                       54
                                   1.0   credit                    1077
                                         debit                  

In [11]:
datas2=train_data[['isFraud','card1','card2','card3','card4','card5','card6']]
print(datas2.groupby(['card1','card2','card3','card4','card5','card6']).sum())

                                                          isFraud
card1 card2 card3 card4            card5 card6                   
1     0.0   1.0   american express 0.0   credit               1.0
                  discover         0.0   credit               0.0
                                   1.0   credit              59.0
                                         debit                0.0
                  mastercard       0.0   credit               0.0
                                         debit                0.0
                                   1.0   credit               0.0
                                         debit                0.0
                  visa             0.0   credit               4.0
                                         debit                1.0
                                   1.0   credit             260.0
                                         debit               59.0
      1.0   1.0   american express 0.0   credit              13.0
          

In [12]:
# 计算不同类别的欺诈率
getcardtype=(datas2.groupby(['card1','card2','card3','card4','card5','card6']).sum().astype(float))/(datas.groupby(['card1','card2','card3','card4','card5','card6']).count()).rename(columns={'TransactionID':"isFraud"})
print((datas2.groupby(['card1','card2','card3','card4','card5','card6']).sum().astype(float))/(datas.groupby(['card1','card2','card3','card4','card5','card6']).count()).rename(columns={'TransactionID':"isFraud"}))

                                                           isFraud
card1 card2 card3 card4            card5 card6                    
1     0.0   1.0   american express 0.0   credit           0.017857
                  discover         0.0   credit           0.000000
                                   1.0   credit           0.047658
                                         debit            0.000000
                  mastercard       0.0   credit           0.000000
                                         debit            0.000000
                                   1.0   credit           0.000000
                                         debit            0.000000
                  visa             0.0   credit           0.011561
                                         debit            0.018519
                                   1.0   credit           0.241411
                                         debit            0.013654
      1.0   1.0   american express 0.0   credit           0.07

> 将card1-card6 转换为type
1. 采用将以上card分类多发、易发、不易发和其他  由以上数据可看出差别较大
2. 类似于时间处理方案

In [13]:
train_data['cardtype'] = getcardtype.loc[[str(train_data['card1'].values),str(train_data['card2'].values),
                                          str(train_data['card3'].values),str(train_data['card4'].values),
                                          str(train_data['card5'].values),str(train_data['card6'].values)]]

TypeError: incompatible index of inserted column with frame index

#### ```遗留问题

In [14]:
# 地址  存在某些地址情况下大部分为欺诈  并且addr1和addr2必然同时存在  可以认为addr1和add2可合并为同一条信息
datas = train_data[['addr1','addr2','isFraud']]
rate = datas.groupby(['addr1','addr2']).sum().astype(float)/datas.groupby(['addr1','addr2']).count()
print(datas.shape)
print(rate.values.shape)
print(np.where(rate.values>0.9)[0].shape)
print(np.where(rate.values<0.0001)[0].shape)

(590540, 3)
(437, 1)
(7,)
(337,)


1. 需要做归一化的列：TransactionAmt
2. 需要做特征重构并归一化：邮箱 时间 ProductCD


> 590540条数据中65706条地址信息为空 如果addr1为空 则addr2为空
524834条数据存在地址信息  且addr1和addr2没有共同的值
将地址信息分为4类  0 缺失地址信息  1 高频易发  2：低频不易发  3：不易发
~ 如此做信息损失会不会有点大  因此采用类似于？？

In [15]:
# 距离信息 如果dist1存在 则 dist2必然为空  反之亦然，由此我们可以将距离信息划分为三大类，dist1距离  dist2距离 距离未知
# 将dist2补全到dist1中组合成为一个特征  空值赋值为-1
train_data['dist1']=train_data['dist1']+train_data['dist2']
train_data.drop(['dist2'],axis=1)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,hours
0,2987000,0,86400,68.500000,W,1,0.0,1.0,discover,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
1,2987001,0,86401,29.000000,W,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
2,2987002,0,86469,59.000000,W,1,1.0,1.0,visa,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
3,2987003,0,86499,50.000000,W,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
4,2987004,0,86506,50.000000,H,1,1.0,1.0,mastercard,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750635
5,2987005,0,86510,49.000000,W,1,1.0,1.0,visa,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
6,2987006,0,86522,159.000000,W,1,1.0,1.0,visa,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
7,2987007,0,86529,422.500000,W,1,1.0,1.0,visa,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635
8,2987008,0,86535,15.000000,H,1,1.0,1.0,visa,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750635
9,2987009,0,86536,117.000000,W,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635


In [16]:
# 邮箱  划分为两条特征  email1  1.出现的邮箱类型特征 （0-1）  2.0：收发件邮箱均为空  1：发件为空 2：收件为空  3：发件收件均不为空

In [17]:
# M1-M9特征合并  M1 M2 M3若为空同时为空 不为空同时不为空；M7 M8 M9同M1 M2 M3；因此我们可将者六条合并为两条；M4-M6排列组合成两条；之后查看
# 对接过的影响再进行操作

In [16]:
#对于V数据的处理
vData = train_data.iloc[:,-340:-1]
vData=vData.fillna(-1)
print(vData.shape)

(590540, 339)


In [17]:
# 对V数据保留  通过PCA降维
from sklearn.decomposition import PCA
pca_sk = PCA(n_components=2)

newMat = pca_sk.fit_transform(vData)  

In [18]:
print(newMat)
print(newMat.shape[1])

[[-6602.02265548  -772.10225946]
 [-6601.83925817  -900.30912777]
 [-6601.83932375  -900.30921212]
 ...
 [-6601.83934608  -900.30923544]
 [-6605.30551094  1150.61041517]
 [-6602.4047432   -518.2594488 ]]
2


In [19]:
x=[i for i in range(-340,-1)]
train_data.drop(train_data.columns[x], axis=1, inplace=True)

In [20]:
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,M1,M2,M3,M4,M5,M6,M7,M8,M9,hours
0,2987000,0,86400,68.5,W,1,0.0,1.0,discover,1.0,...,T,T,T,M2,F,T,NaN,NaN,NaN,0.750635
1,2987001,0,86401,29.0,W,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,0.750635
2,2987002,0,86469,59.0,W,1,1.0,1.0,visa,1.0,...,T,T,T,M0,F,F,F,F,F,0.750635
3,2987003,0,86499,50.0,W,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN,0.750635
4,2987004,0,86506,50.0,H,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635


In [21]:
for i in range(newMat.shape[1]):
    train_data['pcaV'+str(i+1)] = newMat[:,i]
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,M3,M4,M5,M6,M7,M8,M9,hours,pcaV1,pcaV2
0,2987000,0,86400,68.5,W,1,0.0,1.0,discover,1.0,...,T,M2,F,T,NaN,NaN,NaN,0.750635,-6602.022655,-772.102259
1,2987001,0,86401,29.0,W,1,1.0,1.0,mastercard,1.0,...,NaN,M0,T,T,NaN,NaN,NaN,0.750635,-6601.839258,-900.309128
2,2987002,0,86469,59.0,W,1,1.0,1.0,visa,1.0,...,T,M0,F,F,F,F,F,0.750635,-6601.839324,-900.309212
3,2987003,0,86499,50.0,W,1,1.0,1.0,mastercard,1.0,...,NaN,M0,T,F,NaN,NaN,NaN,0.750635,-6605.332332,1551.596976
4,2987004,0,86506,50.0,H,1,1.0,1.0,mastercard,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.750635,163765.483946,-665.244687


In [22]:
# 处理productcd
print(train_data['ProductCD'].unique())

['W' 'H' 'C' 'S' 'R']


In [30]:
train_data['ProductCD']=train_data['ProductCD'].replace('W',1)
train_data['ProductCD']=train_data['ProductCD'].replace('H',2)
train_data['ProductCD']=train_data['ProductCD'].replace('C',3)
train_data['ProductCD']=train_data['ProductCD'].replace('S',4)
train_data['ProductCD']=train_data['ProductCD'].replace('R',5)
print(train_data['ProductCD'].unique())
# 根据不同P类别的productCD的相似度重新对productCD进行标签
ProductCDX = train_data.groupby('ProductCD').mean()['isFraud']
ProductCDsC = train_data.groupby('ProductCD')['isFraud'].count()
ProductCDsX = ProductCDX.values
ProductCDsCs= (ProductCDsX[np.array(train_data['ProductCD'].astype("int").values)]-ProductCDsX.min())/(ProductCDsX.max()-ProductCDsX.min()) 
train_data['ProductCDsCs'] = (ProductCDsX[np.array(train_data['ProductCD'].astype("int").values)]-ProductCDsX.min())/(ProductCDsX.max()-ProductCDsX.min()) 

[1 2 3 4 5]


IndexError: index 5 is out of bounds for axis 0 with size 5

In [28]:
print(train_data['ProductCDsCs'].unique())

[-0.02716549 -0.02716547 -0.02716547 -0.02716544 -0.02716539]
